# 3) Urban Network Analysis (UNA) Tools: Accessibility, Closest Facility and Service Area

We learned how to load and visualize layers in section 1, and learned how to create, diagnose and visualize network in section 2. In this section we wil learn how to use the UNA Acissibility tools. As we learned from the preivos two section, the following code:
* loads the sidewalk, buildinbg entrances and subway entrances geometries.
* Create a topological network
* Create the graph objects. 

This is all we need to start using the UNA tools in Madina.


In [ ]:
import madina as md
cambridge = md.Zonal()

#Loading sidewalks, buildings and subway geometries. 
cambridge.load_layer('sidewalks', 'Cities/Cambridge/Data/sidewalks.geojson')
cambridge.load_layer('buildings', 'Cities/Cambridge/Data/building_entrances.geojson')
cambridge.load_layer('subway', 'Cities/Cambridge/Data/subway.geojson')

# Creating a network, and adding origins and destinations
cambridge.create_street_network(source_layer="sidewalks", node_snapping_tolerance=0.1)
cambridge.insert_node(label='origin', layer_name="subway")
cambridge.insert_node(label='destination', layer_name="buildings")

# Creating graphs
cambridge.create_graph()

In [ ]:
round(254.7)

We first need to import the UNA tools module:

In [ ]:
import madina.una.tools as una

We want to know how many building entrances or people can be reached from subway stations in a 5-minute (300m) walkshed. 

## una.accissibility - Reach Index
Running the `una.accessibility` and giving it parameters `reach=True, search_radius=300` will measure how many destinations (building entrances) are reachable from origins (subway). We see that the northen station could be reached by 106 building entrance, while the southern station could be reached by 112 building entrance. 

The <code>una_accessibility</code> creates a column called <code>una_reach</code> when the parameter <code>reach=True</code>. We could use that to explore more visualization functionalities:

In [ ]:
una.accessibility(
    cambridge,
    reach=True,
    search_radius=300, 
    save_reach_as="reach_to_buildings"
)


cambridge.create_map(
    [
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": "subway", "radius": "reach_to_buildings", 'text':'reach_to_buildings', 'color': [255, 0, 0]},
    ]
)


Setting the parameter <code>weight='people'</code> enables us to weight the destinations (building entrances) by how many people actually live in these building. We could then see that 2,789 people could reach the northen station by walking a maximum of 300 meters. The southern station could be reached by 3,017 people.

In [ ]:
una.accessibility(
    cambridge,
    reach=True,
    search_radius=300,
    weight='people',
    save_reach_as="gravity_to_people"
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": "subway", "radius": "gravity_to_people", 'text':'gravity_to_people', 'color': [255, 0, 0]},
    ]
)

## una.service_area
Sometimes, we want to examine a section of our data in isolation. The `service_area()` function is a great introduction into that. The library internally uses a library called [GeoPandas](https://geopandas.org), where a layer is stored as a GeoDataFrame. GeoDataFrames are an extenstion to the well known [Pandas library's](https://pandas.pydata.org/) DataFrame. If you're familiar with the Pandas Dataframe, you'll be able to use all the functionality you're familiar with. The only difference is that a GeoDataFrame always has a `geometry` column, and supports a wide array of spatial operations.

The `service_area()` function returns:
* `destinations`: a GeoDataFrame  containing all the destinations that are reachable and covered by an origin's service area.
* `network_edges`: a GeoDataframe containing all network segments inside the origin's service area. 
* `scope_gdf`: a Dataframe that contains the boundaries of the service area.

As discussed in section 1, `create_map()` could take either a layer name from the layers we loaded, or a gdf (i.e. GeoDataFrame).

In [ ]:
destinations, network_edges, scope_gdf = una.service_area(
    cambridge,
    search_radius=100,
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'opacity': 0.1},
        {"layer": 'buildings', 'opacity': 0.1},
        {"layer": 'subway'},
        {"gdf": network_edges, "color": [0, 255, 0]},
        {"gdf": destinations, "color": [255, 0, 0]},
        {"gdf": scope_gdf, "color": [0, 0, 255], 'opacity': 0.10},
    ]
)

let's see the impact of imposing a turn penalty on the service area:

In [ ]:
destinations, network_edges, scope_gdf = una.service_area(
    cambridge,
    search_radius=100,
    turn_penalty=True, 
    turn_penalty_amount=25, 
    turn_threshold_degree=45,
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'opacity': 0.1},
        {"layer": 'buildings', 'opacity': 0.1},
        {"layer": 'subway'},
        {"gdf": network_edges, "color": [0, 255, 0]},
        {"gdf": destinations, "color": [255, 0, 0]},
        {"gdf": scope_gdf, "color": [0, 0, 255], 'opacity': 0.10},
    ]
)

We see each station's service area getting smaller in response to imposing a penalty on turns. 

## Setting up Attributes
It is possible to make changes to layers by modifying existing values or adding new attributes. these new or modified attributes could be used as weights in UNA tools. Every object geometry in a layer has an identifier, and each layer once loaded is assigned an `id` attribute. We can visualize these ids

In [ ]:
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'buildings', 'text':'id', 'color': [255, 0, 0]},
    ]
)

We can give building entrances with ids 2 and 115 a new atttribute `students` and assign values 10 and 10, for building ids 2 and 115 respectively. We could do that by accessing the Geodataframe of the layer building, then using [the function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) `at(id_number, attribute_name)` to set individual attributes. 

For visualization, we could use the pandas' [loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) to access specific rows/geometries by specifying a list of their ids. The next map shows building entrances with ids 2 and 115, and shows their `students` attribute which we've set for both to 10

In [ ]:
cambridge.layers['buildings'].gdf.at[2, 'students'] = 10
cambridge.layers['buildings'].gdf.at[115, 'students'] = 10

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"gdf": cambridge['buildings'].gdf.loc[[2, 115]], 'text':'students', 'color': [255, 0, 0]},
    ]
)

From the map below, we now notice that 20 students can reach the northen station, and 10 students could reach he southern station on a 300 meter walk.

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    search_radius=300,
    weight='students', 
    save_reach_as='reach_to_students'
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'subway', "radius": "reach_to_students", 'text':'reach_to_students', 'color': [255, 0, 0]},
    ]
)

## una.accissibility - Gravity Index
The <a href="https://en.wikipedia.org/wiki/Trip_distribution#Gravity_model"> gravity model</a> of transportation is one of many <a href="https://en.wikipedia.org/wiki/Distance_decay">distance decay</a> functions. The aim of a distance decay function (e.g. gravity) is to quantify the inverse relationship between distance, and willingness to make trips. applying gravity peanilize further destinations by reducing their contribution, assume that they are less accissable. For this reason, a gravity index is always less than or equal to a reach index.
<p>

When factoring in a gravity decay, we notice that the northen station only attracts 16.88 students (As opposed to 20) and the southern station now attracts only 8.67 students (as opposed to 10).

In [ ]:
una.accessibility(
    cambridge, 
    gravity=True,
    search_radius=300,
    beta=0.001,
    alpha=1,
    weight='students',
    save_gravity_as='gravity_to_students'
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'subway', "radius": "gravity_to_students", 'text':'gravity_to_students', 'color': [255, 0, 0]},
    ]
)

## Comparing Reach and Gravity of People to Subway
We go back to using `people` as weight, and we again call the function `una_accessibility()`, this time, setting both reach and gravity to `True`. The map below shows the two stations with their grqavity score. You can hover each station to see more information. The northern station has a reach index of 2,789 but a gravity index of only 2,361.54, while the southern station has a reach index of 3,017 and a gravity index of only 2,606.81.

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    gravity=True,
    search_radius=300,
    beta=0.001,
    alpha=1,
    weight='people',
    save_reach_as='reach_to_people', 
    save_gravity_as='gravity_to_people'
)
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'subway', "radius": "gravity_to_people", 'text':'gravity_to_people', 'color': [255, 0, 0]},
    ]
)

## una.closest_facility()
When using <code>una_accessibility()</code>, there is potential for overlap between destinations assigned to each origin, so a destination would be double counted in an origin's accissability indices. The `closest_facility()` assigns each  destination (in this case, building entrance) to one single origin (a subway station in this case)

In [ ]:
una.closest_facility(
    cambridge,
    weight='people',
    gravity=True,
    reach=True,
    search_radius=300,
    beta=0.001, 
    save_reach_as='reach_to_closest_people', 
    save_gravity_as='gravity_to_closest_people', 
    save_closest_facility_as='closest_subway', 
    save_closest_facility_distance_as='distance_to_closest_subway',
)

We can see how destinations are assigned to their closest origin

In [ ]:
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'buildings',"text": "distance_to_closest_subway",  "color_by_attribute": "closest_subway", "color_method": "categorical"},#, 'radius': "una_reach"},
    ]
) 

When destinations are assigned to their closest facility, we can see how that impacts reach and gravity, as origins are splitting destinations between them and a destination is only reachable by its closest origin. For reach:

In [ ]:
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'subway', "radius": "reach_to_closest_people", 'text':'reach_to_closest_people', 'color': [255, 0, 0]},
    ]
)

and for gravity: 

In [ ]:
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'subway', "radius": "gravity_to_closest_people", 'text':'gravity_to_closest_people', 'color': [255, 0, 0]},
    ]
)

Compare how reach and gravity vary when using accissibility and closest facility


<table border="1" cellpadding="2">
<tbody>
<tr>
<td>Function</td>
<td colspan="2"><code>una.accessibility()</code></td>
<td colspan="2"><code>una.closest_facility()</code></td>
</tr>
<tr>
<td>Station</td>
<td>Reach</td>
<td>Graviy</td>
<td>Reach</td>
<td>Gravity</td>
</tr>
<tr>
<td>Northren</td>
<td>2,789.00</td>
<td>2.361.54</td>
<td>1,265.00</td>
<td>1,120.92</td>
</tr>
<tr>
<td>Southren</td>
<td>3,017.00</td>
<td>2,606.81</td>
<td>1,914.00</td>
<td>1,716.57</td>
</tr>
</tbody>
</table>


## The Reverse: Reach and Gravity of Subway to People
To examine the different interpretation of accessibility indices based on what we define as origins and destinations, lets flip our origins from `subway` to `buildings' and destinations from 
`buildings` to `subway`.

In order to do this, we need to first clear up the origins and destination nodes we created earlier, bur we could still use the same neteowk edges (sidewalks) as before, since changing the origins and destinations will not impact the underlying network segments and intersections. Calling the function `clear_nodes()` removes  existing oeigins and destinations. We then need to insert origins and destinations, then create a graph object, just like before.

In [ ]:
cambridge.clear_nodes()
cambridge.insert_node(layer_name="buildings", label='origin')
cambridge.insert_node(layer_name="subway", label='destination')
cambridge.create_graph()

The reach index shown below, shows how many subway stations could be reached from each building enterance, by walking 300 meters. Most building entrances can access two stations, but buildings on the phrepherey, could only access one of the two stations within a 300 meter walk,

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    search_radius=300,
    save_reach_as='reach_to_subway'
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'buildings', "color_by_attribute": "reach_to_subway", 'color_method': 'gradient',  'text':'reach_to_subway'}
    ]
)

We oculd add an attriute called `daily_trains` to each subway station, measure and vosialize the reach accissibility metric

In [ ]:
cambridge.layers['subway'].gdf.at[0, 'daily_trains'] = 20
cambridge.layers['subway'].gdf.at[1, 'daily_trains'] = 50


una.accessibility(
    cambridge, 
    reach=True,
    search_radius=300,
    weight='daily_trains', 
    save_reach_as='reach_to_daily_trains'
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [125, 125, 125]},
        {"layer": 'buildings', "color_by_attribute": "reach_to_daily_trains", 'color_method': 'gradient',  'text':'reach_to_daily_trains'}
    ]
)

We notice some buildings now have access to either 20, 50 or 70 daily trains, depending on what stations they can access within a 300 meter walk. 

As for the gravity index, the interpretation of what these values mean is sensitive to the `beta`` parameter which could translate to the "area's willingness to walk". living in a building with a high gravity index, means that you have more subway options that are close and could easily be reached on foot. A low gravity index, means that you live in a building with fewer subway options that are further out.

In [ ]:
una.accessibility(
    cambridge, 
    gravity=True,
    search_radius=300,
    beta=0.004,
    save_gravity_as='gravity_to_subway'
)
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [135, 125, 125]},
        {"layer": 'buildings', "color_by_attribute": "gravity_to_subway", 'color_method':'gradient', 'text': 'gravity_to_subway'},
    ]
)

When setting `weight='daily_trains'`, and `gravity=True`, each building is assigned a gravity-adjusted number of daily train based on both, how many stations this building can access within a 300 meter walk, and how far are these stations. 

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    gravity=True,
    search_radius=300,
    beta=0.004,
    weight='daily_trains', 
    save_gravity_as='gravity_to_daily_trains'
)
cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks', 'color': [135, 125, 125]},
        {"layer": 'buildings', "color_by_attribute": "gravity_to_daily_trains", 'color_method':'gradient', 'text': 'gravity_to_daily_trains'},
    ]
)


## Saving output into files

for buildings, we've identified the `closest_subway` and the `distance_to_closest_subway`, and measured `reach_to_subway`, `reach_to_daily_trains`, `gravity_to_subway`, `gravity_to_daily_trains`: 

In [ ]:
cambridge['buildings'].gdf.head(5)

And for subway stations, we calculated `reach_to_buildings`, `reach_to_students`, `gravity_to_students`, `reach_to_people`, `gravity_to_people`, `reach_to_closest_people`, `gravity_to_closest_people`, and defined a new attribute `daily_trains`

In [ ]:
cambridge['subway'].gdf.head(5)

We could save these layers to a few file formats, by using the [Geopandas' `to_file()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_file.html) function, which can create multiple file formats. Also, you could use any of [Pandas' output](https://pandas.pydata.org/docs/reference/io.html) functions. Notice that field names/column names in shapefiles cannot exceed 10 characters. 

In [ ]:
# Shapefiles
cambridge['buildings'].gdf.to_file('building_accessibility.shp')
cambridge['subway'].gdf.to_file('subway_accessibility.shp')

# GeoJSON
cambridge['buildings'].gdf.to_file('building_accessibility.geojson', driver='GeoJSON')
cambridge['subway'].gdf.to_file('subway_accessibility.geojson', driver='GeoJSON')

# CSV
cambridge['buildings'].gdf.to_csv('building_accessibility.csv')
cambridge['subway'].gdf.to_csv('subway_accessibility.csv')